# 🎥 Camera Access via Network IP - Security Fix

**Problema**: `navigator.mediaDevices` é `undefined` quando acessado via `http://192.168.100.37:3000`

**Causa**: Chrome bloqueia APIs poderosas (câmera, geolocalização, etc.) em contextos HTTP não-seguros

**Data**: 18 de outubro de 2025

## ❌ Por que está falhando?

### Chrome Security Policy

APIs consideradas "poderosas" (câmera, microfone, geolocalização) **só funcionam em contextos seguros**:

✅ **PERMITIDO**:
- `https://` (qualquer domínio)
- `http://localhost:3000`
- `http://127.0.0.1:3000`

❌ **BLOQUEADO**:
- `http://192.168.100.37:3000` ← Seu caso
- `http://192.168.x.x:*` (qualquer IP local)
- `http://meudominio.com`

### Console Log Evidência
```javascript
✅ mediaDevices API available: false  // ← AQUI ESTÁ O PROBLEMA
❌ Erro: getUserMedia API não disponível neste navegador
```

## 🔧 Solução 1: HTTPS com Certificado Self-Signed (RECOMENDADO)

### Por que HTTPS?
- ✅ Funciona em **qualquer IP** (192.168.x.x, 10.x.x.x, etc.)
- ✅ Funciona em **qualquer dispositivo** da rede
- ✅ Solução **permanente** e **profissional**
- ✅ Simula ambiente de **produção**

### Passos de Implementação

### Passo 1: Gerar Certificado SSL Self-Signed

**No PowerShell (Windows):**

In [ ]:
# Criar pasta para certificados
New-Item -ItemType Directory -Force -Path "./certs"

# Gerar certificado self-signed válido por 365 dias
# Substitua 192.168.100.37 pelo seu IP
$cert = New-SelfSignedCertificate `
    -DnsName "192.168.100.37", "localhost" `
    -CertStoreLocation "cert:\LocalMachine\My" `
    -NotAfter (Get-Date).AddYears(1) `
    -KeyLength 2048 `
    -KeyAlgorithm RSA `
    -HashAlgorithm SHA256 `
    -KeyUsage DigitalSignature, KeyEncipherment `
    -TextExtension @("2.5.29.37={text}1.3.6.1.5.5.7.3.1")

# Exportar certificado (.pfx)
$password = ConvertTo-SecureString -String "academia2025" -Force -AsPlainText
Export-PfxCertificate -Cert $cert -FilePath "./certs/server.pfx" -Password $password

# Exportar chave pública (.pem) para Fastify
$certPath = "cert:\LocalMachine\My\$($cert.Thumbprint)"
Export-Certificate -Cert $certPath -FilePath "./certs/server.crt"

Write-Host "✅ Certificado gerado com sucesso!"
Write-Host "📁 Localização: ./certs/"
Write-Host "🔑 Senha: academia2025"

### Passo 2: Converter .pfx para .pem/.key (Fastify precisa)

**Instalar OpenSSL** (se não tiver):
- Download: https://slproweb.com/products/Win32OpenSSL.html
- Ou via Chocolatey: `choco install openssl`

In [ ]:
# Extrair chave privada (.key)
openssl pkcs12 -in ./certs/server.pfx -nocerts -out ./certs/server.key -nodes -passin pass:academia2025

# Extrair certificado (.pem)
openssl pkcs12 -in ./certs/server.pfx -clcerts -nokeys -out ./certs/server.pem -passin pass:academia2025

Write-Host "✅ Certificados convertidos para formato PEM!"
Get-ChildItem ./certs/

### Passo 3: Configurar Fastify para HTTPS

**Editar `src/server.ts`:**

In [ ]:
import Fastify from 'fastify';
import { readFileSync } from 'fs';
import { join } from 'path';

// ✅ HTTPS Configuration
const isProduction = process.env.NODE_ENV === 'production';
const useHTTPS = process.env.USE_HTTPS === 'true' || !isProduction;

let httpsOptions = {};

if (useHTTPS) {
  try {
    httpsOptions = {
      https: {
        key: readFileSync(join(__dirname, '../certs/server.key')),
        cert: readFileSync(join(__dirname, '../certs/server.pem'))
      }
    };
    console.log('🔒 HTTPS enabled with self-signed certificate');
  } catch (error) {
    console.warn('⚠️ HTTPS certificates not found, falling back to HTTP');
    console.warn('   Run: npm run cert:generate');
  }
}

const fastify = Fastify({
  logger: true,
  ...httpsOptions
});

// Resto do código...
const PORT = process.env.PORT || 3000;
const HOST = process.env.HOST || '0.0.0.0';

fastify.listen({ port: PORT, host: HOST }, (err, address) => {
  if (err) throw err;
  console.log(`🚀 Server listening on ${address}`);
});

### Passo 4: Adicionar Scripts ao package.json

In [ ]:
{
  "scripts": {
    "dev": "USE_HTTPS=true tsx watch src/server.ts",
    "dev:http": "tsx watch src/server.ts",
    "cert:generate": "powershell -ExecutionPolicy Bypass -File ./scripts/generate-cert.ps1",
    "cert:trust": "powershell -ExecutionPolicy Bypass -File ./scripts/trust-cert.ps1"
  }
}

### Passo 5: Confiar no Certificado (Chrome)

**Opção A: Via Script PowerShell (Recomendado)**

In [ ]:
# scripts/trust-cert.ps1

# Importar certificado para Trusted Root
$cert = Get-PfxCertificate -FilePath "./certs/server.pfx"
$store = New-Object System.Security.Cryptography.X509Certificates.X509Store(
    "Root", "CurrentUser"
)
$store.Open("ReadWrite")
$store.Add($cert)
$store.Close()

Write-Host "✅ Certificado adicionado às autoridades confiáveis!"
Write-Host "🔄 Reinicie o navegador para aplicar mudanças"

**Opção B: Manual no Chrome**

1. Acesse `https://192.168.100.37:3000`
2. Clique em **"Avançado"** no aviso de certificado
3. Clique em **"Prosseguir para 192.168.100.37 (não seguro)"**
4. Ou importe o certificado:
   - Chrome → Configurações → Privacidade e Segurança → Segurança
   - **Gerenciar certificados do dispositivo**
   - **Autoridades de Certificação Raiz Confiáveis**
   - **Importar** → Selecione `certs/server.crt`

### Passo 6: Testar HTTPS

In [ ]:
# Reiniciar servidor com HTTPS
npm run dev

# Abrir no navegador
Start-Process "https://192.168.100.37:3000/#checkin-kiosk"

# Console deve mostrar:
# 🔒 HTTPS enabled with self-signed certificate
# 🚀 Server listening on https://0.0.0.0:3000
# ✅ mediaDevices API available: true  ← FUNCIONANDO!

## 🔧 Solução 2: Chrome Flags (TEMPORÁRIO - Apenas Dev)

⚠️ **USE APENAS PARA TESTES** - Não é solução permanente

### Passo 1: Habilitar Flag no Chrome

In [ ]:
1. Abra: chrome://flags/#unsafely-treat-insecure-origin-as-secure

2. Adicione: http://192.168.100.37:3000

3. Selecione: "Enabled"

4. Clique em "Relaunch"

✅ Câmera vai funcionar em http://192.168.100.37:3000

### Desvantagens desta Solução

❌ Precisa configurar **em cada navegador** de cada dispositivo
❌ Configuração **perde ao limpar dados** do Chrome
❌ **Não funciona em produção**
❌ Usuários terão que fazer o mesmo
❌ Não é **seguro** (bypassa proteções do navegador)

## 🔧 Solução 3: Port Forwarding + localhost (WORKAROUND)

### Conceito
Redirecionar conexões de `localhost:3000` para `192.168.100.37:3000`

### Implementação (Windows)

In [ ]:
# PowerShell como Administrador

# Adicionar regra de port forwarding
netsh interface portproxy add v4tov4 `
    listenport=3000 `
    listenaddress=127.0.0.1 `
    connectport=3000 `
    connectaddress=192.168.100.37

# Verificar regra
netsh interface portproxy show all

# Agora acesse: http://localhost:3000/#checkin-kiosk
# ✅ Câmera funcionará!

### Desvantagens

❌ Só funciona **no próprio servidor**
❌ Outros dispositivos da rede **não conseguem acessar**
❌ Tablet precisa estar no mesmo computador (inviável)

## 📊 Comparação das Soluções

| Solução | Funciona em Rede? | Permanente? | Produção? | Complexidade |
|---------|-------------------|-------------|-----------|---------------|
| **HTTPS Self-Signed** | ✅ Sim | ✅ Sim | ✅ Sim* | Média |
| **Chrome Flags** | ✅ Sim | ❌ Não | ❌ Não | Baixa |
| **Port Forwarding** | ❌ Não | ✅ Sim | ❌ Não | Baixa |

*Em produção, use certificado SSL válido (Let's Encrypt, etc.)

## 🎯 Recomendação Final

### Para Seu Cenário (Tablet na Rede Local)

**USE: Solução 1 (HTTPS Self-Signed)**

✅ **Por quê?**
- Funciona em **qualquer dispositivo** da rede
- Tablet pode acessar via `https://192.168.100.37:3000`
- Solução **profissional** e **permanente**
- Simula **ambiente de produção**
- Não precisa configurar cada navegador

### Próximos Passos

1. ✅ Gerar certificado SSL (5 minutos)
2. ✅ Configurar Fastify para HTTPS (5 minutos)
3. ✅ Confiar no certificado (2 minutos)
4. ✅ Testar no navegador desktop
5. ✅ Testar no tablet

**Tempo total**: ~15 minutos

## 🔍 Debugging

### Verificar se HTTPS está funcionando

In [ ]:
// No console do navegador (F12)

// 1. Verificar protocolo
console.log('Protocol:', window.location.protocol); // Deve ser "https:"

// 2. Verificar mediaDevices disponível
console.log('mediaDevices:', navigator.mediaDevices); // Deve ser objeto
console.log('getUserMedia:', typeof navigator.mediaDevices?.getUserMedia); // Deve ser "function"

// 3. Testar acesso à câmera
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream => {
    console.log('✅ Câmera acessada com sucesso!', stream);
    stream.getTracks().forEach(track => track.stop());
  })
  .catch(err => {
    console.error('❌ Erro ao acessar câmera:', err.name, err.message);
  });

### Erros Comuns

**Erro**: `NotAllowedError: Permission denied`
- **Solução**: Clique em "Permitir" quando o navegador pedir permissão

**Erro**: `NotFoundError: Requested device not found`
- **Solução**: Verifique se webcam está conectada e funcionando

**Erro**: `NotReadableError: Could not start video source`
- **Solução**: Feche outros programas usando a câmera (Zoom, Teams, etc.)

**Erro**: `TypeError: Cannot read properties of undefined`
- **Solução**: Protocolo não é HTTPS ou flag não habilitada

## 📚 Referências

- [Chrome Secure Contexts](https://www.chromium.org/Home/chromium-security/prefer-secure-origins-for-powerful-new-features/)
- [MDN - getUserMedia](https://developer.mozilla.org/en-US/docs/Web/API/MediaDevices/getUserMedia)
- [Fastify HTTPS](https://www.fastify.io/docs/latest/Reference/Server/#https)
- [Self-Signed Certificates](https://web.dev/how-to-use-local-https/)